In [21]:
from collections import deque
import copy
import time
import copy
from collections import deque
from constraint import Problem, AllDifferentConstraint
# ---------- AC3 + Backtracking Sol ----------
def read_sudoku_from_file(filename):
    with open(filename, 'r') as file:
        return [line.strip() for line in file if line.strip()]


def parse_puzzle(puzzle_string):
    return [[int(puzzle_string[i * 9 + j]) for j in range(9)] for i in range(9)]


def puzzle_to_string(puzzle):
    return ''.join(str(puzzle[i][j]) for i in range(9) for j in range(9))

# Get domains for unassigned cells
def get_domains(puzzle):
    domains = {}
    for r in range(9):
        for c in range(9):
            if puzzle[r][c] == 0:
                possible = set(range(1, 10))
                for i in range(9):
                    possible.discard(puzzle[r][i])  
                    possible.discard(puzzle[i][c])  
                br, bc = 3 * (r // 3), 3 * (c // 3)
                for i in range(3):
                    for j in range(3):
                        possible.discard(puzzle[br+i][bc+j])  
                domains[(r, c)] = possible
    return domains


def ac3(domains):
    
    queue = deque([(xi, xj) for xi in domains for xj in get_neighbors(xi) if xj in domains])
    while queue:
        xi, xj = queue.popleft()
        if xi not in domains or xj not in domains:
            continue  
        if revise(domains, xi, xj):
            if not domains[xi]:
                return False
            for xk in get_neighbors(xi):
                if xk in domains and xk != xj:
                    queue.append((xk, xi))
    return True

def revise(domains, xi, xj):
    revised = False
    to_remove = set()
    
    for x in domains[xi]:
        if all(x == y for y in domains[xj]):
            to_remove.add(x)
    if to_remove:
        domains[xi] -= to_remove
        revised = True
    return revised

def get_neighbors(cell):
    r, c = cell
    neighbors = set()
    for i in range(9):
        if i != c: neighbors.add((r, i))
        if i != r: neighbors.add((i, c))
    br, bc = 3 * (r // 3), 3 * (c // 3)
    for i in range(3):
        for j in range(3):
            nr, nc = br + i, bc + j
            if (nr, nc) != cell:
                neighbors.add((nr, nc))
    return neighbors



def backtrack(puzzle, domains):
    if not domains:
        return True

    var = select_unassigned_var(domains)
    for value in sorted(domains[var]):
        new_puzzle = copy.deepcopy(puzzle)
        new_domains = copy.deepcopy(domains)
        new_puzzle[var[0]][var[1]] = value
        new_domains.pop(var)
        for neighbor in get_neighbors(var):
            if neighbor in new_domains:
                new_domains[neighbor].discard(value)
        if ac3(new_domains):
            if backtrack(new_puzzle, new_domains):
                puzzle[:] = new_puzzle
                return True
    return False

def select_unassigned_var(domains):
    return min(domains, key=lambda var: len(domains[var]))

# solve ginle sudoku
def solve_sudoku(puzzle):
    domains = get_domains(puzzle)
    if not ac3(domains):
        return None
    if backtrack(puzzle, domains):
        return puzzle
    return None

def solve_puzzles(input_filename, output_filename):
    puzzles = read_sudoku_from_file(input_filename)
    solutions = []

    for puzzle_string in puzzles:
        puzzle = parse_puzzle(puzzle_string)
        solved = solve_sudoku(puzzle)
        if solved:
            solutions.append(puzzle_to_string(solved))
        else:
            solutions.append("No solution")

    with open(output_filename, 'w') as f:
        for line in solutions:
            f.write(line + "\n")

    

input_filename = "sudoku_puzzle.txt"
output_filename = "sudoku_output.txt"
solve_puzzles(input_filename, output_filename)


In [15]:
# ---------- GPT Sol ----------
def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
    br, bc = row - row % 3, col - col % 3
    for i in range(3):
        for j in range(3):
            if board[br + i][bc + j] == num:
                return False
    return True

def brute_force_solver(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num
                        if brute_force_solver(board):
                            return True
                        board[row][col] = 0
                return False
    return True


In [16]:
# ---------- Library Sol ----------
def solve_with_library(puzzle):
    problem = Problem()
    cells = [(i, j) for i in range(9) for j in range(9)]

    for cell in cells:
        r, c = cell
        val = puzzle[r][c]
        if val == 0:
            problem.addVariable(cell, range(1, 10))
        else:
            problem.addVariable(cell, [val])

    for i in range(9):
        problem.addConstraint(AllDifferentConstraint(), [(i, j) for j in range(9)])
        problem.addConstraint(AllDifferentConstraint(), [(j, i) for j in range(9)])

    for br in range(3):
        for bc in range(3):
            block = []
            for i in range(3):
                for j in range(3):
                    block.append((3 * br + i, 3 * bc + j))
            problem.addConstraint(AllDifferentConstraint(), block)

    solution = problem.getSolution()
    if not solution:
        return None

    solved = [[0] * 9 for _ in range(9)]
    for (r, c), val in solution.items():
        solved[r][c] = val
    return solved


In [ ]:
# ---------- Main Comparison ----------
def compare_all_methods(puzzles):
    
    ac3_result = []
    lib_result = []
    bf_result = []
    # Method 1: AC3 Sol
    start = time.time()
    for puzzle_string in puzzles:
        puzzle = parse_puzzle(puzzle_string)
        ac3_board = copy.deepcopy(puzzle)
        ac3_result.append(solve_sudoku(ac3_board))
        
    ac3_time = time.time() - start
    
    # Method 2: GPT Sol
    start = time.time()
    for puzzle_string in puzzles:
        puzzle = parse_puzzle(puzzle_string)
        brute_force_solver(puzzle)
    brute_time = time.time() - start

    # Method 3: Library Sol
    start = time.time()
    for puzzle_string in puzzles:
        puzzle = parse_puzzle(puzzle_string)
        solve_with_library(puzzle)
    lib_time = time.time() - start

    print("=== Comparison Results ===")
    print("\nAC3 + CSP:")
    print("Time: {:.6f}s".format(ac3_time))

    print("\nBrute Force:")
    print("Time: {:.6f}s".format(brute_time))

    print("\nLibrary Constraint Solver:")
    print("Time: {:.6f}s".format(lib_time))
    
# ---------- Run ----------
puzzle_string = read_sudoku_from_file("sudoku_puzzle.txt")
compare_all_methods(puzzle_string)

=== Comparison Results ===

AC3 + CSP:
Time: 0.915527s

Brute Force:
Time: 73.533716s

Library Constraint Solver:
Time: 0.268032s
